<a href="https://colab.research.google.com/github/essential2189/ML_study/blob/main/Hands-On_ML/4.%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC


iris = datasets.load_iris()
X = iris['data'][:, (2, 3)] # 꽃잎 길이, 꽃잎 너비
y = (iris['target'] == 2).astype(np.float64) # iris-virginica

svm_clf = Pipeline([
                    ('scaler', StandardScaler()),
                    ('linear_svc', LinearSVC(C=1, loss='hinge')), # == ('svc', SVC(kernel='linear', C=1))    LinearSVC 가 SVC(kernel='linear') 보다 빠름
                    # 훈련 샘플보다 특성이 많지 않다면 성능을 위해 dual=False 로 해야합니다.
])

svm_clf.fit(X, y)
svm_clf.predict([[5.5, 1.7]])

array([1.])

LinearSVC : 커널 트릭을 지원 안함. 계산 복잡도 $O(m*n)$

SVC : 커널 트릭을 지원. 계란 복잡도 $O(m^2*n)$ ~ $O(m^3*n)$ $->$ 복잡하지만 작거나 중간 규모의 훈련 세트에 잘 맞음. 하지만 특성의 개수에는, 특히 희소 특성(각 샘플에 0이 아닌 특성이 몇 개 없는 경우)인 경우에는 잘 확장됨. 이 경우 알고리즘 성능이 샘플이 가진 0이 아닌 특성의 평균 수에 거의 비례.

SVC, SVR 허용오차 파라미터 ε : tol

SVR 도로의 폭 파라미터 ε : epsilon

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
                               ('poly_features', PolynomialFeatures(degree=3)), # 다항식 특성 추가, 주어진 차수까지 특성 간의 모든 교차항을 추가 (degree=3, 특성 a,b -> a^3, a^2, b^3, b^2, ab, a^2b, ab^2)
                               ('scaler', StandardScaler()),
                               ('svm_clf', LinearSVC(C=10, loss='hinge'))
])

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [4]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
                                ('scaler', StandardScaler()),
                                ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5)), # kernel trick : 실제로는 특성을 추가 하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를 얻을 수 있다.
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [6]:
rbf_kernel_svm_clf = Pipeline([
                               ('scaler', StandardScaler()),
                               ('svm_clf', SVC(kernel='rbf', gamma=5, C=0.001)), # kernel trick : 실제로는 특성을 추가 하지 않으면서 유사도 특성을 많이 추가한 것과 같은 결과를 얻을 수 있다.
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

SVM에서 가장 먼저 선형 커널을 시도해보는게 좋음. 특히 훈련 세트가 아주 크거나 특성 수가 많을 경우.

훈련 세트가 너무 크지 않다면 가우시안 RBF 커널도 시도해볼만함.


In [7]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)

svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [8]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel='poly', degree=2, C=100, epsilon=0.1)

svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

LinearSVR 시간이 훈련 세트 크기에 비례해서 선형적으로 늘어남. (LinearSVC 처럼)

SVR 훈련 세트가 커지면 휠씬 느려짐. (SVC 처럼)